In [1]:
// we need to specify the path and explicitly load the CAF library
#pragma cling add_library_path("/usr/local/lib")
#pragma cling add_include_path("/usr/local/include")
#pragma cling load("/usr/local/lib/libcaf_core.so")

# Polymorphic Actors

This example shows how one can have different implementations of a trait and therefore achieve the polymorphism.


In [2]:
#include <iostream>
#include <string>
#include <caf/all.hpp>

## Define trait

In [3]:
using get_edges_atom = caf::atom_constant<caf::atom("get_edges")>;

// This is our interface definition that
// various actors should implement
using shape_actor =
    caf::typed_actor<caf::replies_to<get_edges_atom>::with<int>>;

## Various implementations of above trait

We will implement shape_actor using the functions (statically typed and event
based).

We have 3 types of actors that implement the same interface.


In [4]:
shape_actor::behavior_type circle_actor_fn() {
  return {[](get_edges_atom) { return 0; }};
};

In [5]:
shape_actor::behavior_type triangle_actor_fn() {
  return {[](get_edges_atom) { return 3; }};
};


In [6]:
shape_actor::behavior_type square_actor_fn() {
  return {[](get_edges_atom) { return 4; }};
};

## An actor that uses the implementations

In [7]:
// Now we build an actor that will be supplied one
// of them and it would query the number of edges
// using that

// As should be clear from the signature of query_shapes
// that it takes the trait as the argument. In other words,
// the caller will provide the implementation

void query_shapes(caf::event_based_actor *self, const shape_actor &shape) {
  self->request(shape, std::chrono::seconds(10), get_edges_atom::value)
      .then([=](const int &edges_count) {
        caf::aout(self) << edges_count << std::endl;
      });
}

## Main / Entrypoint

In [8]:
void caf_main() {
    
  caf::actor_system_config cfg;
  caf::actor_system system{cfg};
  caf::scoped_actor self{system};

  // create all of our shape actors
  auto circle_actor = system.spawn(circle_actor_fn);
  auto triangle_actor = system.spawn(triangle_actor_fn);
  auto square_actor = system.spawn(square_actor_fn);

  system.spawn(query_shapes, circle_actor);
  system.spawn(query_shapes, triangle_actor);
  system.spawn(query_shapes, square_actor);
}

In [9]:
caf_main()

3
0
4
